In [1]:
import os
import cv2
import numpy as np
import torch
from torchvision.transforms.functional import to_tensor
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from scipy.signal import windows  # Hanning 창 활용

In [2]:
# 모델 초기화
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = maskrcnn_resnet50_fpn(weights="DEFAULT")

# Box Predictor 수정
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=2)

# Mask Predictor 수정
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes=2)

# 모델 로드 (CUDA로 모델을 로드)
model_path = r"C:\Users\user\Desktop\Cell DIC Mask R-CNN\mask_genrator\mask_generator\mask_rcnn_final_no compression.pth"
model.load_state_dict(torch.load(model_path, map_location=device))  # `map_location=device`로 CUDA로 로드
model.to(device)  # 모델을 CUDA로 이동
model.eval()

# 경로 설정
input_dir = r"C:\Users\user\Desktop\Cell DIC Mask R-CNN\mask_genrator\mask_generator\DIC_MIP"
output_dir = r"C:\Users\user\Desktop\Cell DIC Mask R-CNN\mask_genrator\mask_generator\Output_Opening_Final"
os.makedirs(output_dir, exist_ok=True)

# 가중치 맵 생성 함수
def create_weight_map(patch_size):
    """
    Hanning 창을 활용한 가중치 맵 생성
    """
    hanning_1d = windows.hann(patch_size)  # Hanning 1D 생성
    weight_map = np.outer(hanning_1d, hanning_1d)  # 2D로 확장
    return weight_map / weight_map.max()  # 정규화 (0~1)

# 패치 분할 및 병합 함수
def process_image(image, patch_size=512, overlap=384, threshold=0.9, kernel_size_opening=50, kernel_size_dilation=7):
    """
    이미지를 패치 단위로 나누어 모델 예측을 수행한 뒤 최종 마스크를 병합하고 연산을 적용
    """
    h, w, _ = image.shape
    step = patch_size - overlap
    mask_full = np.zeros((h, w), dtype=np.float32)
    weight_map_full = np.zeros((h, w), dtype=np.float32)
    patch_weight = create_weight_map(patch_size)  # 패치 크기에 맞는 가중치 맵 생성

    # 슬라이딩 윈도우로 패치 처리 및 병합
    for y in range(0, h - overlap, step):
        for x in range(0, w - overlap, step):
            # 패치 자르기
            patch = image[y:y + patch_size, x:x + patch_size]
            if patch.shape[0] < patch_size or patch.shape[1] < patch_size:
                continue

            # 모델 예측 (CUDA로 이동된 이미지를 사용)
            patch_tensor = to_tensor(patch).unsqueeze(0).to(device)  # 이미지를 CUDA로 이동
            with torch.no_grad():
                prediction = model(patch_tensor)[0]

            if "masks" in prediction and len(prediction["masks"]) > 0:
                pred_mask = prediction["masks"].cpu().numpy()  # CPU로 이동시킨 후 NumPy 배열로 변환
                combined_mask = pred_mask.max(axis=0).squeeze()

                # 병합
                mask_full[y:y + patch_size, x:x + patch_size] += combined_mask * patch_weight
                weight_map_full[y:y + patch_size, x:x + patch_size] += patch_weight

    # 가중치로 나눠서 평균화
    weight_map_full[weight_map_full == 0] = 1  # 0으로 나누는 것을 방지
    mask_full /= weight_map_full

    # 이진화
    binary_mask = (mask_full > threshold).astype(np.uint8) * 255

    # **오프닝 연산 적용**
    kernel_opening = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size_opening, kernel_size_opening))
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel_opening)

    # **Dilation 연산 적용**
    kernel_dilation = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size_dilation, kernel_size_dilation))
    binary_mask = cv2.dilate(binary_mask, kernel_dilation)

    return binary_mask

# 이미지 처리 및 저장
for img_file in os.listdir(input_dir):
    img_path = os.path.join(input_dir, img_file)
    image = cv2.imread(img_path)
    if image is None:
        print(f"Cannot read image: {img_file}")
        continue

    print(f"Processing: {img_file}")
    result_mask = process_image(image)

    # 결과 저장
    output_path = os.path.join(output_dir, f"{os.path.splitext(img_file)[0]}_mask.png")
    cv2.imwrite(output_path, result_mask)
    print(f"Saved: {output_path}")

print("Processing complete!")


Processing: DIC_MIP_10.png


C:\Users\user\anaconda3\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Saved: C:\Users\user\Desktop\Cell DIC Mask R-CNN\mask_genrator\mask_generator\Output_Opening_Final\DIC_MIP_10_mask.png
Processing: DIC_MIP_11.png
Saved: C:\Users\user\Desktop\Cell DIC Mask R-CNN\mask_genrator\mask_generator\Output_Opening_Final\DIC_MIP_11_mask.png
Processing: DIC_MIP_12.png
Saved: C:\Users\user\Desktop\Cell DIC Mask R-CNN\mask_genrator\mask_generator\Output_Opening_Final\DIC_MIP_12_mask.png
Processing: DIC_MIP_13.png
Saved: C:\Users\user\Desktop\Cell DIC Mask R-CNN\mask_genrator\mask_generator\Output_Opening_Final\DIC_MIP_13_mask.png
Processing: DIC_MIP_14.png
Saved: C:\Users\user\Desktop\Cell DIC Mask R-CNN\mask_genrator\mask_generator\Output_Opening_Final\DIC_MIP_14_mask.png
Processing: DIC_MIP_15.png
Saved: C:\Users\user\Desktop\Cell DIC Mask R-CNN\mask_genrator\mask_generator\Output_Opening_Final\DIC_MIP_15_mask.png
Processing: DIC_MIP_16.png
Saved: C:\Users\user\Desktop\Cell DIC Mask R-CNN\mask_genrator\mask_generator\Output_Opening_Final\DIC_MIP_16_mask.png
Proce